In [141]:
import numpy as np
import math
import operator
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
data = pd.read_csv('泰坦尼克号数据.csv')
data = data.replace('',np.NaN)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [142]:

data = data.drop(columns='Cabin')
data = data.drop(columns='PassengerId')
data = data.drop(columns='Name')
data = data.drop(columns='Ticket')
data = data.drop(columns='Fare')
null_all = data.isnull().sum()
null_all

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [143]:
data["Age"] = data.fillna(data["Age"].mean())
null_all = data.isnull().sum()
null_all

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    2
dtype: int64

In [144]:
data.iloc[:,2]

0        male
1      female
2      female
3      female
4        male
        ...  
886      male
887    female
888    female
889      male
890      male
Name: Sex, Length: 891, dtype: object

LabelEncoder


In [145]:
from sklearn.preprocessing import LabelEncoder

data.iloc[:,2] = LabelEncoder().fit_transform(data.iloc[:,2])
data.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,1,0,1,0,S
1,1,1,0,1,1,0,C
2,1,3,0,1,0,0,S
3,1,1,0,1,1,0,S
4,0,3,1,0,0,0,S


In [146]:
for i in data.columns:
    if i!= "Embarked":
        data[i] = (data[i] - min(data[i])) / (max(data[i])-min(data[i]))

In [147]:
new_data = data.copy()
def loaddata(data,label):
    datax = data[data[[label]].isnull().T.any()]
    for i in datax.index:
        data = data.drop(i)
    return data , datax




In [148]:
data , datax = loaddata(data,'Embarked')
ans = 0
datax.iloc[0,-1]

nan

In [149]:
def getNeighbors(trainingSet, testInstance, k):
    answer = {}
    for j in range(testInstance.shape[0]):
        distances = []
        distance0 = 0
        for x in range(trainingSet.shape[0]):
            for n in range(trainingSet.shape[1]):
                if n != trainingSet.shape[1]-1:
                    distance0 += (trainingSet.iloc[x,n]-testInstance.iloc[j,n])**2
            dist = (distance0)**0.5
            distances.append([trainingSet.iloc[x,:], dist])
        distances.sort(key = lambda x :x[-1])
        neighbors = []
        for x in range(k):
            neighbors.append(distances[x]) 
        classVotes = {}
        for i in range (len(neighbors)):
            response = neighbors [x][0][-1]
            if response in classVotes:
                classVotes[response] += 1
            else:
                classVotes[response] = 1
        sortedVotes = sorted(classVotes.items(), reverse=True)
        answer[j]=sortedVotes[0][0]
    return answer

In [150]:
neighbors=getNeighbors(data,datax,4)
neighbors[0]
for i in range(datax.shape[0]):
   datax.iloc[i,-1]=neighbors[i]
data = pd.concat([data,datax])
index01= np.arange(data.shape[0])
data.index = index01
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0.0,1.0,1.0,0.0,0.125,0.000000,S
1,1.0,0.0,0.0,1.0,0.125,0.000000,C
2,1.0,1.0,0.0,1.0,0.000,0.000000,S
3,1.0,0.0,0.0,1.0,0.125,0.000000,S
4,0.0,1.0,1.0,0.0,0.000,0.000000,S
...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,0.125,0.333333,S
887,1.0,0.0,1.0,1.0,0.000,0.000000,C
888,0.0,1.0,1.0,0.0,0.000,0.000000,Q
889,1.0,0.0,0.0,1.0,0.000,0.000000,S


In [151]:
x=data.iloc[:,[-1]]
x

,Embarked
0,S
1,C
2,S
3,S
4,S
...,...
886,S
887,C
888,Q
889,S


In [152]:
enc = OneHotEncoder(categories="auto").fit(x)
result = enc.transform(x).toarray()
result

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [153]:
pd.DataFrame(enc.inverse_transform(result))

,0
0,S
1,C
2,S
3,S
4,S
...,...
886,S
887,C
888,Q
889,S


In [154]:
enc.get_feature_names()

array(['x0_C', 'x0_Q', 'x0_S'], dtype=object)

In [155]:
new_data = pd.concat([data,pd.DataFrame(result)],axis=1)
new_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,0,1,2
0,0.0,1.0,1.0,0.0,0.125,0.0,S,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.125,0.0,C,1.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.000,0.0,S,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.125,0.0,S,0.0,0.0,1.0
4,0.0,1.0,1.0,0.0,0.000,0.0,S,0.0,0.0,1.0


In [156]:
new_data.drop(["Embarked"],axis=1,inplace=True)
new_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,0,1,2
0,0.0,1.0,1.0,0.0,0.125,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.125,0.0,1.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.000,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.125,0.0,0.0,0.0,1.0
4,0.0,1.0,1.0,0.0,0.000,0.0,0.0,0.0,1.0


In [157]:
new_data.columns=['Survived','Pclass','Sex','Age','SibSp','Parch','Embark_C','Embark_Q','Embark_S']
new_data.head()
data = new_data

In [158]:

# add a ones column - this makes the matrix multiplication work out easier
data.insert(data.shape[1], 'Ones', 1)
# set X (training data) and y (target variable)
x = data.iloc[0:800,:]
x_test = data.iloc[800:891,1:data.shape[1]]
y_test = data.iloc[800:891,[0]]
theta = np.ones((x.shape[1]-1,1))
x = np.matrix(x).astype('float')
x_test = np.matrix(x_test).astype('float')
y_test = np.matrix(y_test).astype('float')


In [159]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [160]:
def cost(theta, x):
    y = x[:,0]
    x = x[:,1:]
    first = np.multiply(-y, np.log(sigmoid(x * theta)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(x * theta)))
    return np.sum(first - second) / (len(x))

In [161]:
cost(theta, x)

2.327083189060314

In [162]:
def sgdgradient(theta, x,alpha,stop_grad,stop_cost):
    times = 0
    delta = 1000000
    while times < stop_grad and abs(delta) >stop_cost:
        for j in range(theta.shape[0]):
            cost0 = cost(theta,x)
            np.random.shuffle(x)
            x01 = x[0:1,1:]
            y01 = x[0:1,0]
            error = (sigmoid(x01 * theta)-y01)
            term  = error.T*x01[:,j]
            theta[j] = theta[j] - alpha*(term / len(theta))
            cost1 = cost(theta,x)
            delta = cost0 - cost1 
            times += 1
    
    return theta

In [163]:
theta = sgdgradient(theta, x,1,1000000,0.000001)
theta

array([[-0.91391556],
       [-0.89372275],
       [ 4.24305568],
       [ 0.60523273],
       [ 0.91600372],
       [ 0.38854721],
       [ 0.57357838],
       [-0.21386139],
       [-1.35035574]])

In [164]:
cost(theta, x)

0.08878415889785363

In [165]:
ans=np.zeros((x_test.shape[0],1))

In [166]:
def predict(theta, x_test):
    probability = sigmoid(x_test * theta)
    ans=np.zeros((x_test.shape[0],1))
    for i in range(ans.shape[0]):
        ans[i] = [1 if probability[i] >= 0.5 else 0 ]
    return ans

In [167]:
y_predict = predict(theta,x_test)

In [168]:
num = 0
for i in range(y_predict.shape[0]):
    if y_predict[i] == y_test[i]:
        num += 1
score = num/(y_predict.shape[0]) * 100
score

100.0